# **Notebook BoW**

Instalamos las librerías necesarias en el kernel:

In [1]:
import pandas as pd
import spacy as spc
import nltk
import emoji
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

Importamos el archivo *.csv* al dataframe *texto*:

In [2]:
texto = pd.read_csv(f'C:/Users/Black/Documents/Jupyter/Tareas/df_mini_HS.csv', na_values=r'\N')

print(texto)

    label                                               text
0       1  ESAS COSAS Y OTRAS PUEDEN PASAR POR MANTENER A...
1       1  28: te amodio, odio a la perra de tu amiga ☺️☺...
2       1  @LaDivinaDiva Callate maldita perra. O seguro ...
3       1  @MarysabelPuerto Mejor callate cara de puta o ...
4       1  @xarita327 @TRIKYHUMOR @yonier2012 @casTa1326 ...
5       1              @CocotePR @ashleyhonohan callate puta
6       1  Y el inmigrante recibe ayuda del rico Estado l...
7       1  De los moros no se puede esperar nada bueno, y...
8       1  ¿Por que si a una mujer le pegan un tiro en la...
9       1  Analicemos esto: ¿Si te pones unos shorts así,...
10      0  "See... Tal vez les recordo como Peron protegi...
11      0  PIETRAPIERCE STORY: PURS SANGS ARABES STARS DE...
12      0  ¿Qué dice este de frivolizar el acoso escolar?...
13      0  #Suiza 🇨🇭 retira el permiso a 189 refugiados q...
14      0  Hoy quiero denunciaaaaaaar A LA GENTE PUTO GUA...
15      0  Redomicilie s

eliminamos los caracteres especiales, menciones, hashtags y todo aquél texto que no nos interesa analizar de la columna *texto['text']*:

In [3]:
texto['text'] = texto['text'].str.lower() #Cambiar texto a minúsculas
texto['text'] = texto['text'].str.replace(r'@\S+', '', regex = True).astype(str)  # Eliminar menciones a usuarios
texto['text'] = texto['text'].str.replace(r'http[s]?\://\S+', '', regex = True).astype(str)  # Eliminar enlaces
texto['text'] = texto['text'].str.replace(r'#\S+', '', regex = True).astype(str)  # Eliminar hashtags
texto['text'] = texto['text'].str.replace(r'[0-9]', '', regex = True).astype(str)  # Eliminar números
texto['text'] = texto['text'].str.replace(r'(\(.*\))|(\[.*\])', '', regex = True).astype(str)  # Eliminar paréntesis y corchetes
texto['text'] = texto['text'].str.replace(r'\n', '', regex = True).astype(str)  # Eliminar caracteres de nueva línea
texto['text'] = texto['text'].str.replace(r'(http[s]?\://\S+)|([\[\(].*[\)\]])|([#@]\S+)|\n', '', regex = True).astype(str)  # Eliminar varios patrones
texto['text'] = texto['text'].str.replace(r'(\.)|(,)|(\:)|(\;)', '', regex = True).astype(str)  # Eliminar puntos y comas
texto['text'] = texto['text'].str.replace(r'[¡!]|[¿?]|(\*)|(\-)|(\")|(\/)', '', regex = True).astype(str)  # Eliminar signos de especiales
texto['text'] = texto['text'].apply(lambda s: emoji.replace_emoji(s, '')) #Eliminar emojis

texto['text']

0     esas cosas y otras pueden pasar por mantener a...
1      te amodio odio a la perra de tu amiga pero tú...
2      callate maldita perra o seguro eres un pobre ...
3      mejor callate cara de puta o reputa como tu m...
4                                          cállate puta
5                                          callate puta
6     y el inmigrante recibe ayuda del rico estado l...
7     de los moros no se puede esperar nada bueno y ...
8     por que si a una mujer le pegan un tiro en la ...
9     analicemos esto si te pones unos shorts así en...
10    see tal vez les recordo como peron protegio a ...
11    pietrapierce story purs sangs arabes stars des...
12    qué dice este de frivolizar el acoso escolar p...
13      retira el permiso a  refugiados que fueron d...
14    hoy quiero denunciaaaaaaar a la gente puto gua...
15    redomicilie su sociedad offshore en emiratos á...
16     basta poned pie en pared a tanta provocación ...
17    semana de la juventud torneo futbol   futb

Creamos la lista *lista_tokens*, la cual tendrá como entradas las listas de tokens que se forman de cada comentario:

In [4]:
lista_texto = texto['text'].astype(str).tolist()
lista_tokens = [sub.split() for sub in lista_texto]

Filtramos de cada lista de tokens a todas aquellas palabras que se catalogan como *stopwords*:

In [5]:
spanish_stopwords = stopwords.words('spanish')
palabras_filtradas = [[word for word in list if word not in spanish_stopwords] for list in lista_tokens]
print(palabras_filtradas)

[['cosas', 'pueden', 'pasar', 'mantener', 'inmigracion', 'ilegal', 'españa', 'expulsarlos', 'agresión', 'manteros', 'turista', 'pone', 'colau', 'aprieto', 'vía'], ['amodio', 'odio', 'perra', 'amiga', 'vida', 'tijerazo'], ['callate', 'maldita', 'perra', 'seguro', 'pobre', 'marico', 'detras', 'user', 'femenino', 'chavista', 'colaborador'], ['mejor', 'callate', 'cara', 'puta', 'reputa', 'madre', 'nota', 'hacen', 'falta'], ['cállate', 'puta'], ['callate', 'puta'], ['inmigrante', 'recibe', 'ayuda', 'rico', 'ladrón', 'olvida', 'nacionales', 'mientras', 'viola', 'mata', 'pintada', 'verdad', 'medias'], ['moros', 'puede', 'esperar', 'bueno', 'dicen', 'propios', 'árabes', 'luchado'], ['si', 'mujer', 'pegan', 'tiro', 'cabeza', 'dura', 'tres', 'dias', 'morirse', 'tres', 'dias', 'bala', 'encuentra', 'cerebro'], ['analicemos', 'si', 'pones', 'shorts', 'así', 'calle', 'esperas', 'digan', 'acoso', 'provocación'], ['see', 'tal', 'vez', 'recordo', 'peron', 'protegio', 'eichmann', 'cientos', 'nazis', 'co

Lematizamos cada una de las listas de tokens filtradas, y guardamos estas oraciones en un arreglo de oraciones lematizadas:

In [6]:
nlp = spc.load("es_core_news_sm")
lemita = [" ".join(sentence) for sentence in palabras_filtradas]
lema = [nlp(sentence) for sentence in lemita]
oraciones_lematizadas = [" ".join([token.lemma_ for token in sentence]) for sentence in lema]
oraciones_lematizadas

['cosa poder pasar mantener inmigracion ilegal españa expulsar él agresión mantero turista poner colau aprieto vía',
 'amodio odio perra amigo vida tijerazo',
 'callate maldito perra seguro pobre marico detras user femenino chavista colaborador',
 'mejor callate cara puta reputa madre nota hacer falta',
 'cállate puto',
 'callate puto',
 'inmigrante recibir ayuda rico ladrón olvidar nacional mientras viola mata pintado verdad medio',
 'moro poder esperar bueno decir propio árabe luchado',
 'si mujer peguir tiro cabeza durar tres dia morir él tres dia bala encontrar cerebro',
 'analicer si pón shorts así calle esperas decir acoso provocación',
 'see tal vez recordo peron protegio eichmann ciento nazi conto arab chiste arar garcho camello',
 'pietrapiercir story purs sang arab stars des festivit agar m',
 'decir frivolizar acoso escolar favor queréis dejar decir semejante tontería',
 'retirar permiso refugiado vacación país vía',
 'hoy quiero denunciaaaaaaar gente puto guarra oler sudor 

  
Ahora vectorizamos nuestro arreglo de oraciones lematizadas y generamos el vocabulario utilizado resultante:

In [7]:
vectorizador = CountVectorizer()
vectores = vectorizador.fit_transform(oraciones_lematizadas)
vocabulario = vectorizador.get_feature_names_out()

Por último, creamos un dataframe con nuestro *Bag of Words (vocabulario)*, en donde cada entrada representa la cantidad de veces que fue utilizada una palabra del vocabulario dentro del respectivo comentario, y agregamos la columna *Clase* al final del dataframe:

In [8]:
df_BoW = pd.DataFrame.sparse.from_spmatrix(vectores,columns = vocabulario)
df_BoW.join(texto['label'].rename('Clase'))

,acoso,agar,agresión,amigo,amodio,analicer,aprieto,arab,arar,así,...,vez,vida,viola,voolka,vía,yogurín,you,árabe,él,Clase
0,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,1
1,0,0,0,1,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
9,1,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
